In [1]:
import pandas as pd 
import numpy as np 

import os 
from glob import glob 

from PIL import Image
import albumentations as A
from albumentations import ImageOnlyTransform
from augraphy import *

from tqdm import tqdm

pre_path = '/kkh/'
train_kr_image_path = pre_path + 'data/train_kr'
train_kr_aug_image_path = pre_path + 'data/train_kr_aug'
meta_kr_csv_path = pre_path + 'data/meta_kr.csv'
train_kr_csv_path = pre_path + 'data/train_kr.csv'
meta_kr_df = pd.read_csv(pre_path + 'data/meta_kr.csv')
train_kr_df = pd.read_csv(pre_path + 'data/train_kr.csv')

In [2]:
# train_kr_aug_image_path 폴더 생성
try:
    os.makedirs(train_kr_aug_image_path)
except Exception as e:
    print(f"An error occurred: {e}")

In [6]:
# 수평 뒤집기 변환 (확률 100%)
horizontal_flip = A.Compose([
    A.HorizontalFlip(p=1),
])

# 수직 뒤집기 변환 (확률 100%)
vertical_flip = A.Compose([
    A.VerticalFlip(p=1),
])

# 수평 및 수직 뒤집기 변환 (확률 100%)
double_flip = A.Compose([
    A.HorizontalFlip(p=1),
    A.VerticalFlip(p=1),
])

# 전치(transpose) 변환 (확률 100%)
transpose = A.Compose([
    A.Transpose(p=1)
])

# 전치 후 수평 뒤집기 변환 (확률 100%)
transpose_hflip = A.Compose([
    A.Transpose(p=1), 
    A.HorizontalFlip(p=1),
])

# 전치 후 수직 뒤집기 변환 (확률 100%)
transpose_vflip = A.Compose([
    A.Transpose(p=1),
    A.VerticalFlip(p=1),
])

# 전치 후 수평 및 수직 뒤집기 변환 (확률 100%)
transpose_dflip = A.Compose([
    A.Transpose(p=1),  
    A.HorizontalFlip(p=1),
    A.VerticalFlip(p=1),
])

# 패턴 생성기 객체를 초기화합니다.
pattern = PatternGenerator(
    imgx=120,                # 생성할 패턴 이미지의 가로 크기
    imgy=120,                # 생성할 패턴 이미지의 세로 크기
    n_rotation_range=(10, 15),  # 회전 범위 (10도에서 15도 사이)
    alpha_range=(0.1, 0.2),    # 알파(투명도) 범위
    color=(130, 130, 130),    # 패턴의 색상 (RGB)
)

# 패턴 생성 변환 클래스 정의
class PatternGeneratorTransform(ImageOnlyTransform):
    def __init__(self, pattern_generator, always_apply=False, p=1.0):
        super().__init__(always_apply, p)
        self.pattern_generator = pattern_generator

    def apply(self, img, **params):
        # 이미지에 패턴 생성기를 적용하여 변환된 이미지를 반환합니다.
        return self.pattern_generator(img)

# 다양한 이미지 변환을 조합한 복합 변환 정의
noise_rotate = A.Compose([
    # 80% 확률로 다음 중 하나를 적용:
    A.OneOf([
        A.GaussNoise(var_limit=(100, 900), per_channel=False, p=1),  # 가우시안 노이즈 추가
        PatternGeneratorTransform(pattern, p=1)  # 패턴 생성 변환
    ], p=0.8),
    
    # 50% 확률로 다음 중 하나를 적용:
    A.OneOf([
        A.MotionBlur(p=0.2),                      # 모션 블러
        A.MedianBlur(blur_limit=3, p=0.2),        # 미디언 블러
        A.Blur(blur_limit=(1, 3), p=1),           # 블러 (1부터 3까지의 범위)
    ], p=0.5),

    # 밝기와 대비를 조정하는 변환 (확률 20%)
    A.RandomBrightnessContrast(p=0.2),

    # 색조, 채도, 명도를 조정하는 변환 (확률 20%)
    A.HueSaturationValue(p=0.2),

    # 이미지를 흑백으로 변환 (확률 30%)
    A.ToGray(p=0.3),

    # 이미지를 이동, 스케일, 회전 변환 (확률 50%)
    A.ShiftScaleRotate(
        shift_limit=0.2,      # 이동 범위
        scale_limit=0,        # 스케일 범위 (0은 스케일 없음)
        rotate_limit=30,      # 회전 범위 (최대 30도)
        p=0.5,                # 변환 적용 확률
        border_mode=0,        # 변환 후 경계 처리 방식 (0은 상수 값으로 채우기)
        value=[255, 255, 255] # 경계 영역의 상수 값 (흰색)
    ),
])

In [7]:
# 여러 이미지 변환을 정의한 리스트입니다. 각 변환은 튜플로 되어 있으며, 튜플의 첫 번째 요소는 변환의 접두사(prefix)이고, 두 번째 요소는 변환 객체입니다.
base_aug_types = [
    ("hf_", horizontal_flip),       # 수평 뒤집기 변환, 접두사 "hf_"
    ("vf_", vertical_flip),         # 수직 뒤집기 변환, 접두사 "vf_"
    ("df_", double_flip),           # 수평 및 수직 뒤집기 변환, 접두사 "df_"
    ("t_", transpose),              # 전치 변환, 접두사 "t_"
    ("thf_", transpose_hflip),      # 전치 후 수평 뒤집기 변환, 접두사 "thf_"
    ("tvf_", transpose_vflip),      # 전치 후 수직 뒤집기 변환, 접두사 "tvf_"
    ("tdf_", transpose_dflip)       # 전치 후 수평 및 수직 뒤집기 변환, 접두사 "tdf_"
]


In [8]:
# 빈 리스트를 초기화합니다. 변환된 이미지의 ID와 타겟 정보를 저장할 것입니다.
ids = []
targets = []

# `train_kr_df` DataFrame의 각 행에 대해 반복합니다.
# `itertuples()`는 DataFrame을 튜플 형태로 반복할 수 있게 해줍니다.
for index, ID, target in tqdm(train_kr_df.itertuples(), total=train_kr_df.shape[0], desc='Image augmentation'):
    # 이미지의 파일 경로를 생성합니다.
    image_path = os.path.join(train_kr_image_path, ID)
    
    # 이미지 파일을 열고, numpy 배열로 변환합니다.
    image = np.array(Image.open(image_path))
    
    # `base_aug_types`에 정의된 각 변환에 대해 반복합니다.
    for prefix, aug_function in base_aug_types:
        # 변환 함수를 사용하여 이미지를 변환합니다.
        transformed_image = aug_function(image=image)['image']
        
        # 변환된 이미지의 새로운 ID를 생성합니다.
        new_ID = prefix + ID
        
        # 새로운 ID와 타겟 정보를 리스트에 추가합니다.
        ids.append(new_ID)
        targets.append(target)
        
        # 변환된 이미지를 저장합니다.
        Image.fromarray(transformed_image).save(os.path.join(train_kr_aug_image_path, new_ID))
        
# 변환된 이미지의 ID와 타겟 정보를 담은 DataFrame을 생성합니다.
aug_data = {
    'ID': ids,
    'target': targets
}
aug_data = pd.DataFrame(aug_data)    

# 원본 DataFrame `train_kr_df`와 증강된 데이터 `aug_data`를 결합하여 새로운 DataFrame `df`를 만듭니다.
df = pd.concat([train_kr_df, aug_data])

Image augmentation: 100%|██████████| 1570/1570 [00:22<00:00, 68.51it/s]


In [9]:
df.to_csv(pre_path + 'data/train_kr_aug.csv', index=False)